<a href="https://colab.research.google.com/github/tourihasi/Openstudio/blob/main/ifcopenshell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install ifcopenshell eppy geomeppy shapely


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 101.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 61.4 MB/s eta 0:00:00


In [ ]:
from google.colab import files

print("IFCファイルを選択してください")
uploaded_ifc = files.upload()

print("Energy+.idd を選択してください")
uploaded_idd = files.upload()


IFCファイルを選択してください


Saving シミュレーション導入.ifc to シミュレーション導入.ifc
Energy+.idd を選択してください


Saving Energy+.idd to Energy+.idd


In [ ]:
# -*- coding: utf-8 -*-
"""
IfcSpace の三角メッシュを
1) 屋根/壁/床に分類（高さ+法線）
2) 同一平面の三角形を統合して1枚の平面ポリゴン化
3) BUILDINGSURFACE:DETAILED として IDF に出力
"""
import io, os, math, sys
import numpy as np

# 依存ライブラリ
try:
    import ifcopenshell, ifcopenshell.geom as ifc_geom
    import ifcopenshell.util.element as ifc_elem
except Exception as e:
    raise RuntimeError("ifcopenshell が未インストールです。先に `pip install ifcopenshell` を実行してください。") from e
try:
    from shapely.geometry import Polygon, MultiPolygon
    from shapely.ops import unary_union
except Exception as e:
    raise RuntimeError("shapely が未インストールです。先に `pip install shapely` を実行してください。") from e
try:
    from eppy.modeleditor import IDF
except Exception as e:
    raise RuntimeError("eppy が未インストールです。先に `pip install eppy` を実行してください。") from e

# ========= 入力 =========
EPLUS_IDD = "/content/Energy+.idd"   # すでにアップロード済み
IFC_PATH  = "/content/input.ifc"     # すでにアップロード済み
IDF_OUT   = "/content/out_merged.idf"

assert os.path.isfile(EPLUS_IDD), f"Energy+.idd が見つかりません: {EPLUS_IDD}"
assert os.path.isfile(IFC_PATH),  f"IFC が見つかりません: {IFC_PATH}"

# ========= IDF 初期化 =========
IDF.setiddname(EPLUS_IDD)
idf = IDF(io.StringIO("Version,25.1;"))   # ← IDDに合わせて 24.1 等へ変更可
idf.newidfobject("SITE:LOCATION", Name="Tokyo", Latitude=35.68, Longitude=139.76, Time_Zone=9, Elevation=10.0)
idf.newidfobject("TIMESTEP", Number_of_Timesteps_per_Hour=6)
idf.newidfobject("BUILDING", Name="IFC_Import_Merged", North_Axis=0.0, Terrain="City",
                 Loads_Convergence_Tolerance_Value=0.04, Temperature_Convergence_Tolerance_Value=0.4)

# ========= Construction（簡易 NoMass） =========
def ensure_nomass_construction(cat, u):
    R = 1.0/max(u,1e-6)
    mat = idf.newidfobject("MATERIAL:NOMASS",
                           Name=f"{cat}_NoMass_R{R:.3f}",
                           Roughness="MediumRough", Thermal_Resistance=R,
                           Thermal_Absorptance=0.9, Solar_Absorptance=0.7, Visible_Absorptance=0.7)
    return idf.newidfobject("CONSTRUCTION", Name=f"{cat}_U{u:.3f}", Outside_Layer=mat.Name)

CON_WALL  = ensure_nomass_construction("Wall",  1.50)
CON_ROOF  = ensure_nomass_construction("Roof",  0.80)
CON_FLOOR = ensure_nomass_construction("Floor", 1.20)

# ========= ifc 読み込み & geom 設定 =========
ifc = ifcopenshell.open(IFC_PATH)
settings = ifc_geom.settings()
def set_opt(s, key, val):
    try:
        s.set(getattr(s, key), val)
    except Exception:
        pass
set_opt(settings, "USE_WORLD_COORDS", True)
set_opt(settings, "INCLUDE_CURVES", False)
set_opt(settings, "DISABLE_TRIANGULATION", False)

# ========= 幾何ユーティリティ =========
def normalize(v):
    n = np.linalg.norm(v)
    return v / n if n else v

def tri_normal(p0, p1, p2):
    return normalize(np.cross(p1 - p0, p2 - p0))

def plane_key(p, n, tol=1e-5):
    """同一平面をクラスタするためのキー（法線方向とオフセットを丸める）"""
    n = normalize(n)
    # 向きの符号は気にしない（統合のため）、Z優先の符号正規化
    if n[2] < 0: n = -n
    d = -np.dot(n, p)  # 平面: n·x + d = 0
    return (round(n[0],5), round(n[1],5), round(n[2],5), round(d,5))

def build_plane_basis(n):
    """平面の直交基底 (u, v) を作る"""
    n = normalize(n)
    # n にほぼ平行でないベクトルを選んで u を作る
    a = np.array([0.0,0.0,1.0]) if abs(n[2]) < 0.9 else np.array([1.0,0.0,0.0])
    u = normalize(np.cross(n, a))
    v = normalize(np.cross(n, u))
    return u, v

def project_to_plane_coords(points3d, origin3d, u, v):
    """3D点群を、平面座標 (u,v) 系の 2D へ射影"""
    # 平面上の原点 origin3d を基準にする
    P = np.array(points3d) - origin3d
    xs = P @ u
    ys = P @ v
    return np.column_stack([xs, ys])

def lift_from_plane_coords(coords2d, origin3d, u, v):
    """2D平面座標から 3D へ戻す"""
    xs, ys = coords2d[:,0], coords2d[:,1]
    return origin3d + np.outer(xs, u) + np.outer(ys, v)

# ========= 屋根/壁/床の分類しきい値 =========
Z_ROOF_TILT = 0.3
Z_FLOOR_TILT = -0.3
TOP_EPS = 0.20
BOT_EPS = 0.20

def classify_surface(p0, p1, p2, z_top, z_bot):
    """三角面を Roof / Floor / Wall に分類"""
    n = tri_normal(p0, p1, p2)
    zmean = (p0[2]+p1[2]+p2[2])/3.0
    if zmean >= z_top - TOP_EPS and n[2] > 0:  return "Roof"
    if zmean <= z_bot + BOT_EPS and n[2] < 0:  return "Floor"
    if   n[2] >=  Z_ROOF_TILT:  return "Roof"
    elif n[2] <=  Z_FLOOR_TILT: return "Floor"
    else:                       return "Wall"

def con_for(stype):
    return {"Roof": CON_ROOF.Name, "Floor": CON_FLOOR.Name}.get(stype, CON_WALL.Name)

# ========= 三角形 → 平面統合 → IDF 出力 =========
spaces = ifc.by_type("IfcSpace")
print("Total IfcSpace:", len(spaces))

surf_count_before = 0
surf_count_after  = 0

for i, sp in enumerate(spaces, start=1):
    zone_name = sp.Name or f"Space_{i}"
    idf.newidfobject("ZONE", Name=zone_name)

    try:
        shp = ifc_geom.create_shape(settings, sp)
    except Exception as e:
        print(f"[WARN] shape failed for {zone_name}: {e}")
        continue

    verts = np.array(shp.geometry.verts, dtype=float).reshape((-1, 3))
    faces = np.array(shp.geometry.faces, dtype=int).reshape((-1, 3))
    if verts.size == 0 or faces.size == 0:
        continue

    z_top = np.quantile(verts[:,2], 0.95)
    z_bot = np.quantile(verts[:,2], 0.05)

    # 1) 三角形を stype (Roof/Floor/Wall) と 平面キーでグループ化
    groups = {}  # (stype, plane_key) -> list of triangles(3x3)
    for (a,b,c) in faces:
        p0, p1, p2 = verts[a], verts[b], verts[c]
        stype = classify_surface(p0, p1, p2, z_top, z_bot)
        n = tri_normal(p0, p1, p2)
        key = (stype, plane_key(p0, n))
        groups.setdefault(key, []).append(np.vstack([p0,p1,p2]))
    surf_count_before += len(faces)

    # 2) グループごとに 2D へ投影 → union でポリゴン化 → 3D に戻す → IDF 出力
    for (stype, pkey), tris in groups.items():
        tris = np.array(tris)  # (T,3,3)
        # 平面の原点: 最初の三角の1点
        origin = tris[0,0,:]
        # 法線は最初の三角から
        n = tri_normal(tris[0,0,:], tris[0,1,:], tris[0,2,:])
        if n[2] < 0:  # 外向き優先（上向き）に合わせる
            n = -n
        u, v = build_plane_basis(n)

        # 2D 三角形ポリゴン群
        poly_list = []
        for tri in tris:
            coords2d = project_to_plane_coords(tri, origin, u, v)
            poly = Polygon(coords2d)
            if not poly.is_valid:
                poly = poly.buffer(0)
            if poly.area > 1e-6:
                poly_list.append(poly)

        if not poly_list:
            continue

        merged = unary_union(poly_list)  # Polygon or MultiPolygon
        geoms = [merged] if isinstance(merged, Polygon) else list(merged.geoms)

        for gi, poly in enumerate(geoms, start=1):
            # 外周のみ採用（穴は今回は無視。必要なら poly.interiors を fenestration 等へ応用可）
            coords2d = np.array(poly.exterior.coords[:-1])  # 最後は始点重複なので落とす
            # 3D に戻す
            coords3d = lift_from_plane_coords(coords2d, origin, u, v)

            # 頂点が3未満は無視
            if len(coords3d) < 3:
                continue

            name = f"{zone_name}:{stype}:{pkey[1]:.3f}_{gi}"
            con  = con_for(stype)
            bc   = "Ground" if stype=="Floor" else "Outdoors"

            # BUILDINGSURFACE:DETAILED 作成
            fld = dict(
                Name=name, Surface_Type=stype, Construction_Name=con, Zone_Name=zone_name,
                Outside_Boundary_Condition=bc, Sun_Exposure="SunExposed", Wind_Exposure="WindExposed",
                View_Factor_to_Ground="autocalculate", Number_of_Vertices=len(coords3d)
            )
            # 頂点を順に埋める
            for vidx, (x,y,z) in enumerate(coords3d, start=1):
                fld[f"Vertex_{vidx}_Xcoordinate"] = float(x)
                fld[f"Vertex_{vidx}_Ycoordinate"] = float(y)
                fld[f"Vertex_{vidx}_Zcoordinate"] = float(z)

            idf.newidfobject("BUILDINGSURFACE:DETAILED", **fld)
            surf_count_after += 1

print(f"Triangles(before): {surf_count_before}  ->  Merged planar surfaces(after): {surf_count_after}")
idf.saveas(IDF_OUT)
print("Saved:", IDF_OUT)


Total IfcSpace: 14
Triangles(before): 486  ->  Merged planar surfaces(after): 167
Saved: /content/out_merged.idf
